In [1]:
from numba import njit
from numpy import zeros
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [3]:
import os
import csv

In [4]:
from pypde import pde_solver

In [5]:
# flux and source terms
# power-law fluid
n_coeff = 0.20
beta_coeff = 2.0*(1.0+2.0*n_coeff)/(2.0+3.0*n_coeff)
alpha_coeff = 4.0 # Fr=1.0/(alpha_coeff)**0.50
L_x = 6.5913
dist_amp = 0.01


def F(Q):
    F_ = zeros(2)

    h = Q[0]
    q = Q[1]

    F_[0] = q
    F_[1] = (beta_coeff*q**2/h)+(0.50*alpha_coeff*h**2)

    return F_


def S(Q):

    S_ = zeros(2)

    h = Q[0]
    q = Q[1]

#     S_[1] = (1.0/beta_coeff)*(h-alpha_coeff*np.sign(up)-2.0*up/(3.0*(h-q/up)))
    S_[1] = h-(q/(h**2))**n_coeff

    return S_

In [6]:
# initial conditions
from numpy import array

@njit
def disturbed_depth(x, normal_depth, wl, amp):
    return (normal_depth*(1.0+amp*np.sin(np.pi*2.0*x/wl)))
    

nx = 500
L = [L_x]
tf = 40.0

delta_x = L_x/nx
normal_depth = 1.0
normal_vel = 1.0

Q_normal = array([normal_depth, normal_depth*normal_vel])

Q0 = zeros([nx, 2])

for i in range(nx):
    x_coord = (i-0.50)*delta_x
    Q0[i] = Q_normal
    Q0[i, 0] = disturbed_depth(x_coord, normal_depth, L_x, dist_amp)

In [7]:
# main simulation program
# del out

# out = pde_solver(Q0, tf, L, F=F, S=S, stiff=True, flux='roe', order=3)
out = pde_solver(Q0, tf, L, F=F, S=S, boundaryTypes='periodic', cfl=0.333, order=2, stiff=False, flux='roe', ndt=81, nThreads=1)

compiling functions...


In [8]:
# Animation 

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(18,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)
# plt.tight_layout()
# fig.subplots_adjust(hspace=8)


# Set the labels
ax1.set_xlabel('x')
ax1.set_ylabel(r'$\tilde{h}$')
ax1.set_xlabel('x')
ax2.set_ylabel(r'$\tilde{q}$')


# Axis limits and lines
line_u=[]
for ax in [ax1, ax2]:
  ax.set_xlim(0.0, L_x)
  ax.set_ylim(0.0, 2.0)
  line_u.append(ax.plot([], [], linewidth=1, color='b', marker='o', markersize=2)[0])

plt.subplots_adjust(bottom=0.1, right=0.75, top=0.8, left = 0.07)
plt.rcParams.update({'font.family': 'Times New Roman','font.size':19})

# animation function.  This is called sequentially
# j0 = slice(2,-2)
arr_size = np.size(out[0,:,0])
x_array = np.linspace(0, L_x, num=arr_size)
def animate(i):
    h = out[i,:,0]
    q = out[i,:,1]
    line_u[0].set_data(x_array, h)
    line_u[1].set_data(x_array, q)

plt.close()
anim = animation.FuncAnimation(fig, animate, frames=75, interval=101, blit=False);
HTML(anim.to_html5_video())

In [9]:
np.shape(out)

(81, 500, 2)

In [10]:
# text files output
un_shape = np.shape(out)
frames = un_shape[0]
arr_size = np.size(out[0,:,0])
x_array = np.linspace(0, L_x, num=arr_size)
for i in range(0, frames):
    t = 1.0*i
    format_string_time = f"{t:.2f}"
    file_name = 'outXYZ_%s' % format_string_time
    with open(file_name, 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(np.transpose(x_array),np.transpose(out[i,:,0])))